# Load, Clean & Tokenize## 🎯 Concept PrimerText preprocessing: lowercase, remove punctuation, tokenize into words.**Expected:** Cleaned text, token lists, sentence length stats

## 📋 Objectives1. Lowercase all text2. Remove punctuation/digits3. Tokenize sentences into words4. Compute sentence length statistics

## 🔧 Setup

In [ ]:
# TODO 1: Import libraries# import pandas as pd# import re# import numpy as np

## 🧹 Clean Text### TODO 2: Lowercase and remove punctuation**Expected:** Cleaned text without special characters

In [ ]:
# TODO 2: Clean text# def clean_text(text):#     text = text.lower()#     text = re.sub(r'[^a-zA-Z\s]', '', text)#     return text# # df['text_clean'] = df['text'].apply(clean_text)

## 🔤 Tokenize### TODO 3: Split into words**Expected:** List of tokens per document

In [ ]:
# TODO 3: Tokenize# df['tokens'] = df['text_clean'].apply(lambda x: x.split())# df['token_count'] = df['tokens'].apply(len)

## 📊 Length Statistics### TODO 4: Analyze sentence lengths**Questions:** Mean/median length? Need padding/truncation?

In [ ]:
# TODO 4: Length stats# print(df['token_count'].describe())# plt.hist(df['token_count'], bins=50)# plt.show()

## 🤔 Reflection1. What's a good max length for padding?2. Any pre-processing issues?

**Your reflection:***Write here*

## 📌 Summary✅ Text cleaned  ✅ Tokens created  ✅ Length stats computed**Next:** `03_vocab_encoding_padding.ipynb`